In [8]:
#final working code
import cv2
import numpy as np
import face_recognition
import os
import winsound  # For playing sound on Windows
from tensorflow.keras.models import load_model

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Load the pre-trained age prediction model
age_net = cv2.dnn.readNetFromCaffe(
    r'C:\Users\kaliv\Downloads\ikkada\deploy_age.prototxt', 
    r'C:\Users\kaliv\Downloads\ikkada\age_net.caffemodel')

# Initialize some variables
font = cv2.FONT_HERSHEY_SIMPLEX
age_list = ['0-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65', '66-70', '71-75', '76-80', '81-85', '86-90', '91-95', '96-100']

# Tolerance for face recognition (adjust as needed)
tolerance = 0.5

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Function to predict age
def predict_age(face_img):
    blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    age_net.setInput(blob)
    age_preds = age_net.forward()
    print("Age predictions:", age_preds)  # Add this line for debugging
    age_index = age_preds[0].argmax()
    age = age_list[age_index]
    return age

# Function to collect known faces and their names
def collect_known_faces():
    known_face_encodings = []
    known_face_names = []

    # Example: Collect known faces from images
    known_faces_dir = r"C:\Users\kaliv\Downloads\ikkada\known_faces\v3"
    for person_dir in os.listdir(known_faces_dir):
        person_path = os.path.join(known_faces_dir, person_dir)
        if os.path.isdir(person_path):
            for filename in os.listdir(person_path):
                image_path = os.path.join(person_path, filename)
                if os.path.isfile(image_path):
                    image = face_recognition.load_image_file(image_path)
                    face_encodings = face_recognition.face_encodings(image)
                    if len(face_encodings) > 0:
                        encoding = face_encodings[0]
                        known_face_encodings.append(encoding)
                        known_face_names.append(person_dir)

    return known_face_encodings, known_face_names

# Load known faces and their names
known_face_encodings, known_face_names = collect_known_faces()

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Encode the face for recognition
        face_encodings = face_recognition.face_encodings(frame, [(y, x+w, y+h, x)])
        
        if len(face_encodings) > 0:
            # Face recognized
            for face_encoding in face_encodings:
                # Compare face encoding with known face encodings
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=tolerance)
                
                if True in matches:
                    # Known face recognized, get the name
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                    
                    # Display name
                    cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    break
                else:
                    # Unknown face, set name as "Unknown"
                    name = "Unknown"
                    
                    # Predict age, gender, and emotion
                    age = predict_age(face)
                    gender = predict_gender(face)
                    emotion = predict_emotion(face)

                    # Display name, age, gender, and emotion in red
                    cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, f'Age: {age}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, f'Gender: {gender}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    
                    # Play buzzer sound
                    winsound.Beep(1000, 500)  # Frequency: 1000Hz, Duration: 500ms
                    
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Age predictions: [[1.3931113e-02 9.5813280e-01 1.9418711e-02 7.2094298e-04 4.0547457e-03
  3.4448267e-03 1.6329861e-04 1.3353450e-04]]
1/1 [==============================] - 1s 576ms/step
Age predictions: [[0.01162452 0.12363149 0.06315622 0.03865992 0.73663306 0.01691287
  0.00403971 0.00534224]]
1/1 [==============================] - 0s 29ms/step
Age predictions: [[0.00655784 0.30978107 0.40845454 0.10092364 0.15248726 0.01594444
  0.00301755 0.00283365]]
1/1 [==============================] - 0s 25ms/step
Age predictions: [[3.0446102e-04 2.2714406e-01 7.5362128e-01 1.5048539e-02 2.8357191e-03
  6.1931595e-04 2.5309063e-04 1.7356584e-04]]
1/1 [==============================] - 0s 28ms/step
Age predictions: [[0.00280145 0.01904663 0.01867907 0.01224985 0.6334572  0.27709782
  0.0298369  0.00683111]]
1/1 [==============================] - 0s 29ms/step
Age predictions: [[0.00153491 0.01270274 0.01367837 0.00622608 0.7368691  0.208538
  0.01692159 0.00352909]]
1/1 [======================

In [9]:
pip list


Package                           Version
--------------------------------- ------------
absl-py                           2.1.0
aiobotocore                       2.7.0
aiohttp                           3.9.3
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.12
altair                            5.0.1
anaconda-anon-usage               0.4.3
anaconda-catalogs                 0.2.0
anaconda-client                   1.12.3
anaconda-cloud-auth               0.1.4
anaconda-navigator                2.5.2
anaconda-project                  0.11.1
anyio                             4.2.0
appdirs                           1.4.4
archspec                          0.2.1
argon2-cffi                       21.3.0
argon2-cffi-bindings              21.2.0
arrow                             1.2.3
astroid                           2.14.2
astropy                           5.3.4
asttokens                         2.0.5
astunparse               

In [2]:
#findings in a txt file
import cv2
import numpy as np
import face_recognition
import os
import winsound  # For playing sound on Windows
from tensorflow.keras.models import load_model

# Load pre-trained models
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
gender_model = cv2.dnn.readNet(r"C:\Users\kaliv\Downloads\ikkada\deploy_gender.prototxt", r"C:\Users\kaliv\Downloads\ikkada\gender_net.caffemodel")
emotion_model = load_model(r"C:\Users\kaliv\Downloads\ikkada\emotion_model.hdf5")

# Load the pre-trained age prediction model
age_net = cv2.dnn.readNetFromCaffe(
    r'C:\Users\kaliv\Downloads\ikkada\deploy_age.prototxt', 
    r'C:\Users\kaliv\Downloads\ikkada\age_net.caffemodel')

# Initialize some variables
font = cv2.FONT_HERSHEY_SIMPLEX
age_list = ['0-5', '6-10', '11-15', '16-20', '21-25', '26-30', '31-35', '36-40', '41-45', '46-50', '51-55', '56-60', '61-65', '66-70', '71-75', '76-80', '81-85', '86-90', '91-95', '96-100']

# Tolerance for face recognition (adjust as needed)
tolerance = 0.5

# Function to predict gender
def predict_gender(face):
    blob = cv2.dnn.blobFromImage(face, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    gender_model.setInput(blob)
    gender = "Male" if gender_model.forward()[0][0] > 0.5 else "Female"
    return gender

# Function to predict emotion
def predict_emotion(face):
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    face = cv2.resize(face, (64, 64))  # Resize to (64, 64)
    face = face.astype("float") / 255.0
    face = np.expand_dims(face, axis=0)
    face = np.expand_dims(face, axis=-1)

    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    prediction = emotion_model.predict(face)
    return emotion_labels[np.argmax(prediction)]

# Function to predict age
def predict_age(face_img):
    blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    age_net.setInput(blob)
    age_preds = age_net.forward()
    print("Age predictions:", age_preds)  # Add this line for debugging
    age_index = age_preds[0].argmax()
    age = age_list[age_index]
    return age

# Function to collect known faces and their names
def collect_known_faces():
    known_face_encodings = []
    known_face_names = []

    # Example: Collect known faces from images
    known_faces_dir = r"C:\Users\kaliv\Downloads\ikkada\known_faces\v3"
    for person_dir in os.listdir(known_faces_dir):
        person_path = os.path.join(known_faces_dir, person_dir)
        if os.path.isdir(person_path):
            for filename in os.listdir(person_path):
                image_path = os.path.join(person_path, filename)
                if os.path.isfile(image_path):
                    image = face_recognition.load_image_file(image_path)
                    face_encodings = face_recognition.face_encodings(image)
                    if len(face_encodings) > 0:
                        encoding = face_encodings[0]
                        known_face_encodings.append(encoding)
                        known_face_names.append(person_dir)

    return known_face_encodings, known_face_names

# Function to write detected information to a text file
def write_to_file(file_path, data):
    with open(file_path, 'a') as file:
        file.write(data + '\n')

# Load known faces and their names
known_face_encodings, known_face_names = collect_known_faces()

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Initialize counter for serial number
sno = 1

# Path to the text file
output_file = "detected_info.txt"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Encode the face for recognition
        face_encodings = face_recognition.face_encodings(frame, [(y, x+w, y+h, x)])
        
        if len(face_encodings) > 0:
            # Face recognized
            for face_encoding in face_encodings:
                # Compare face encoding with known face encodings
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=tolerance)
                
                if True in matches:
                    # Known face recognized, get the name
                    first_match_index = matches.index(True)
                    name = known_face_names[first_match_index]
                    
                    # Display name
                    cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    # Write to file
                    data = f"{sno}\t{name}\t-\t-\t-"
                    write_to_file(output_file, data)
                    sno += 1
                    break
                else:
                    # Unknown face, set name as "Unknown"
                    name = "Unknown"
                    
                    # Predict age, gender, and emotion
                    age = predict_age(face)
                    gender = predict_gender(face)
                    emotion = predict_emotion(face)

                    # Display name, age, gender, and emotion in red
                    cv2.putText(frame, f'Name: {name}', (x, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, f'Age: {age}', (x, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, f'Gender: {gender}', (x, y+h+80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    cv2.putText(frame, f'Emotion: {emotion}', (x, y+h+110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    
                    # Write to file
                    data = f"{sno}\t{name}\t{age}\t{gender}\t{emotion}"
                    write_to_file(output_file, data)
                    sno += 1

                    # Play buzzer sound
                    winsound.Beep(1000, 500)  # Frequency: 1000Hz, Duration: 500ms
                    
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Age predictions: [[1.7296404e-03 9.9105436e-01 6.7758630e-03 2.0873347e-04 2.1531276e-04
  1.0367410e-05 4.8781162e-06 7.9406004e-07]]
1/1 [==============================] - 1s 541ms/step
Age predictions: [[5.1612593e-04 3.7318486e-04 2.3746195e-03 1.0637466e-03 2.1622980e-01
  4.6680889e-01 3.0621681e-01 6.4168195e-03]]
1/1 [==============================] - 0s 26ms/step
Age predictions: [[0.07110147 0.0678347  0.03512841 0.01313391 0.35626152 0.298916
  0.14142507 0.0161989 ]]
1/1 [==============================] - 0s 27ms/step
Age predictions: [[2.7165512e-05 4.9599406e-05 7.5724511e-04 6.4830424e-04 7.5172669e-01
  2.1972717e-01 2.6207371e-02 8.5655076e-04]]
1/1 [==============================] - 0s 27ms/step
Age predictions: [[2.2461267e-05 3.6473051e-05 7.4472820e-04 7.2811928e-04 7.2742182e-01
  2.5237080e-01 1.7883800e-02 7.9181982e-04]]
1/1 [==============================] - 0s 27ms/step
Age predictions: [[9.14004486e-05 1.20785255e-04 9.80999554e-04 8.98142229e-04
  5.8338743